<a href="https://colab.research.google.com/github/vasanramani/Great-Learning/blob/main/AppleHBR/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install -q langchain_community \
              langchain \
              langchain_openai \
              langchain-chroma \
              pymupdf==1.26.3 \
              tiktoken==0.9.0 \
              datasets==4.0.0 \
              evaluate==0.4.5

In [2]:
# Import core libraries
import os
import json
import requests
import chromadb

# Import libraries for working with PDFs and OpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
from IPython.display import Markdown, display

# Import libraries for processing dataframes and text
import tiktoken
import pandas as pd

# Import LangChain components for data loading, chunking, embedding, and vector DBs
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from datasets import Dataset
from langchain_openai import ChatOpenAI

modelToUse="gpt-4o-mini"
maxTokens=500
temp=0
topP=0.95

In [3]:
# Display Markdown formatted string in the notebook output.
# This function takes a string as input and uses IPython.display.Markdown
# to render it as Markdown, providing a more visually appealing output.
def printmd(string):
    display(Markdown(string))

In [4]:
# Identify the environment to switch the file paths either to use Google Drive or Local
import sys
# Check if the operating system is Windows
if sys.platform.startswith('win'):
    printmd("Running on Windows")
    resourcePath=""
else:
    # Assume it's a Linux-based environment, likely Google Colab
    printmd("Running on Linux most likely Colab")
    # Mount Google Drive for Colab environments to access files
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    # Define the base path for resources in Google Drive
    resourcePath="/content/drive/MyDrive/ColabAssignments/AppleHBR/"

Running on Windows

In [5]:
# Load Configuration for OPEN API

file_name = resourcePath + "config.json"
with open(file_name, 'r') as file:
    config = json.load(file)
    OPENAI_API_KEY = config.get("OPENAI_API_KEY")
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")

# Store API credentials in environment variables for use by LangChain and OpenAI clients
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE

In [6]:
# Initialize OpenAI client
openAIClient = OpenAI()

In [7]:
def getDefaultResponse(user_prompt):
    # Access global variables for model configuration (model, max tokens, temperature, top_p)
    global modelToUse, maxTokens, temp, topP
    # Call the OpenAI API to get a chat completion
    completion = openAIClient.chat.completions.create(
        model=modelToUse, # Specify the AI model to use for the completion
        messages=[
            # Define the user's message to the AI model
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=maxTokens, # Set the maximum number of tokens in the response
        temperature=temp,     # Control the randomness of the output (lower = more deterministic)
        top_p=topP            # Control diversity via nucleus sampling (fraction of tokens to consider)
    )
    # Extract and return the content of the AI's response
    return completion.choices[0].message.content

In [8]:
def getGuidedResponse(system_prompt, user_prompt):
    # Access global variables for model configuration
    global modelToUse, maxTokens, temp, topP
    # Call the OpenAI API to get a chat completion with a system prompt for guidance
    completion = openAIClient.chat.completions.create(
        model=modelToUse, # Specify the AI model to use for the completion
        messages=[
            {"role": "system", "content": system_prompt}, # Define the system's role and guiding prompt
            {"role": "user", "content": user_prompt}     # Define the user's message to the AI model
        ],
        max_tokens=maxTokens, # Set the maximum number of tokens in the response
        temperature=temp,     # Control the randomness of the output
        top_p=topP            # Control diversity via nucleus sampling
    )
    # Extract and return the content of the AI's response
    return completion.choices[0].message.content

In [9]:
# Load the PDF from the local location and verify the content
pdfPath = resourcePath + "HBR_How_Apple_Is_Organized_For_Innovation.pdf"
pdfLoader = PyMuPDFLoader(pdfPath)
pdfContent = pdfLoader.load()

In [10]:
# Split the text as multiple chunks
textSplitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=2000, #Complete the code to define the chunk size
    chunk_overlap= 200 #Complete the code to define the chunk overlap
)
pdfChunks = pdfLoader.load_and_split(textSplitter)
pdfChunksLength = len(pdfChunks)
printmd(f"The PDF File is split into {pdfChunksLength} Chunks")

The PDF File is split into 11 Chunks

In [11]:
# Initialize the Embeddings and verify if it loaded properly and it is of the same size
embeddingModel = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE
)
embedding0 = embeddingModel.embed_query(pdfChunks[0].page_content)
embedding1 = embeddingModel.embed_query(pdfChunks[1].page_content)

isEmbeddingSameSize = len(embedding0) == len(embedding1)
printmd(f"Dimension of the embedding vector is {len(embedding0)}")
printmd(f"Both Embeddings are {"SAME" if isEmbeddingSameSize else "DIFFERNT"} size")

Dimension of the embedding vector is 1536

Both Embeddings are SAME size

In [12]:
# Create the vector database from the chunks
vectorDBDir="/content/vectordb"
if not os.path.exists(vectorDBDir):
    os.makedirs(vectorDBDir)
    printmd("Vector DB Directory is created")
vectorDB = Chroma.from_documents(
    documents=pdfChunks,
    embedding=embeddingModel,
    persist_directory=vectorDBDir
)
printmd("Vector DB contents is created")

Vector DB contents is created

In [13]:
# Load the vector DB
vectorDBStore = Chroma(
    embedding_function=embeddingModel,
    persist_directory=vectorDBDir
)
vectorDBStore.embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000023D67A44590>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000023D67A44EC0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://aibe.mygreatlearning.com/openai/v1', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [14]:
vectorDBStore.similarity_search("Who are the authors of this article and who published this article?", k=1)

[Document(id='7bdefca5-d03f-40d2-8679-e34a6d5ce09f', metadata={'author': '', 'moddate': '2020-12-01T18:37:49+00:00', 'file_path': 'HBR_How_Apple_Is_Organized_For_Innovation.pdf', 'trapped': '', 'format': 'PDF 1.6', 'producer': 'Adobe PDF Library 15.0 (via http://bfo.com/products/pdf?version=2.23.5-r33279)', 'subject': '', 'creationDate': "D:20201005141842-04'00'", 'modDate': 'D:20201201183749Z', 'total_pages': 11, 'creationdate': '2020-10-05T14:18:42-04:00', 'page': 1, 'keywords': '', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'title': '', 'encryption': 'Standard V2 R3 128-bit RC4', 'source': 'HBR_How_Apple_Is_Organized_For_Innovation.pdf'}, page_content='2\nHarvard Business Review\nNovember–December 2020\nThis article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.')]

In [15]:
# Initialize a retriever from the Chroma vector store
# This retriever will be used to fetch relevant document chunks based on a query.
vectorDBRetriever = vectorDBStore.as_retriever(
    search_type="similarity", # Specify that the search type should be 'similarity' to find semantically similar documents
    search_kwargs={"k": 1}    # Configure the search to return the top 1 most similar document chunk
)

In [16]:
def getRAGResponse(system_prompt, user_input,k=3):
    global qaUserMsgTemplate, modelToUse, maxTokens, temp, topP
    # Retrieve relevant document chunks using the vector database retriever
    matchingChunks = vectorDBRetriever.invoke(user_input)
    # Extract the page content from the retrieved document chunks
    contexts = [d.page_content for d in matchingChunks]
    # Combine the extracted contexts into a single complete context string
    completeContext = ". ".join(contexts)
    # Populate the user message template with the retrieved context and user's question
    userMessage = qaUserMsgTemplate.replace('{context}', completeContext)
    userMessage = userMessage.replace('{question}', user_input)
    # Generate the response using the OpenAI client
    try:
        response = openAIClient.chat.completions.create(
        model=modelToUse,   # Specify the model to be used for completion
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": userMessage}
        ],
        max_tokens=maxTokens,
        temperature=temp,
        top_p=topP
        )
        # Extract and print the generated text from the response
        response = response.choices[0].message.content.strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

In [17]:
def getResponse(system_prompt, user_question):
    global qaUserMsgTemplate # Ensure global access if not defined in function scope

    printmd(f"## Question: {user_question}")
    printmd(f"### System Prompt: {system_prompt}")

    printmd("### Default Response:")
    default_response = getDefaultResponse(user_question)
    if "JSON" in system_prompt or "XML" in system_prompt:
        print(default_response)
    else:
        printmd(default_response)

    printmd("### Guided Response (Default Prompt):")
    guided_response_default = getGuidedResponse(system_prompt, user_question)
    if "JSON" in system_prompt or "XML" in system_prompt:
        print(guided_response_default)
    else:
        printmd(guided_response_default)

    printmd("### RAG Response (Default Prompt):")
    rag_response_default = getRAGResponse(system_prompt, user_question)
    if "JSON" in system_prompt or "XML" in system_prompt:
        print(rag_response_default)
    else:
        printmd(rag_response_default)

    printmd("\n---\n") # Separator for clarity

In [18]:
question0 = "Who are the authors of this article and who published this article?"
question1 = "List down the three leadership characteristics in bulleted points and explain each one of the characteristics under two lines."
question2 = "Can you explain specific examples from the article where Apple's approach to leadership has led to successful innovations?"
sysPromptDefault="Your response must be friendly and informative."
sysPromptJSON="Your response must be friendly and informative. Create response in a JSON Format."
sysPromptXML="Your response must be friendly and informative. Create response in a XML Format."
qaUserMsgTemplate="### {context} $$$$"

# Below is the call to the getResponse method with the three different questions and different system prompts.
# The system prompts contains prompts to format the contents as default, JSON and XML   
# The output is documented as a well formatted markdown document.

getResponse(sysPromptDefault, question0)
getResponse(sysPromptDefault, question1)
getResponse(sysPromptDefault, question2)

getResponse(sysPromptJSON, question0)
getResponse(sysPromptJSON, question1)
getResponse(sysPromptJSON, question2)

getResponse(sysPromptXML, question0)
getResponse(sysPromptXML, question1)
getResponse(sysPromptXML, question2)

## Question: Who are the authors of this article and who published this article?

### System Prompt: Your response must be friendly and informative.

### Default Response:

I'm sorry, but I can't access external content or databases to retrieve specific articles or their authors. If you provide me with the title or some details about the article, I might be able to help you with general information or context related to the topic.

### Guided Response (Default Prompt):

...

### Guided Response (Default Prompt):

```xml
<response>
    <greeting>Hello!</greeting>
    <information>
        <introduction>
            Apple's approach to leadership has been pivotal in driving successful innovations throughout its history. Here are a few specific examples that highlight this.
        </introduction>
        <examples>
            <example>
                <title>iPhone Launch</title>
                <description>
                    Under Steve Jobs' leadership, the launch of the iPhone in 2007 revolutionized the smartphone industry. Jobs emphasized a focus on design and user experience, which led to a product that combined a phone, an iPod, and an internet communicator into one device. This innovative approach not only transformed Apple's product line but also set new standards for mobile technology.
                </description>
            </example>
            <example>
                <title>Apple Retail Stores</title>
                <description>
                    Apple's retail strategy,

### RAG Response (Default Prompt):

```xml
<response>
    <greeting>Hello!</greeting>
    <information>
        <summary>
            This excerpt highlights Apple's unique organizational culture, emphasizing the importance of specialization and expertise among its leaders. Steve Jobs believed that managers should be experts in their fields, fostering an environment where world-class talent can thrive and learn from one another.
        </summary>
        <keyPoints>
            <point>
                <title>Expert Leadership</title>
                <description>Apple's leaders, like Roger Rosner, exemplify the principle of experts leading experts, ensuring that specialized knowledge is concentrated within teams to enhance innovation and problem-solving.</description>
            </point>
            <point>
                <title>Attention to Detail</title>
                <description>Apple's commitment to detail is evident in product design, such as the unique "squircle" shape of their devices, which requires precise


---


In [19]:
getResponse(sysPromptDefault, "What is an example of the attention to detail?")

## Question: What is an example of the attention to detail?

### System Prompt: Your response must be friendly and informative.

### Default Response:

Attention to detail refers to the ability to notice and consider the small elements that contribute to the overall quality of a task or project. An example of attention to detail can be seen in the work of a graphic designer. 

For instance, when creating a logo, a designer might meticulously choose the exact shade of color to ensure it aligns with the brand's identity, adjust the spacing between letters for optimal readability, and ensure that the design is balanced and visually appealing. They might also double-check for any typos in the text and ensure that the logo looks good in various sizes and formats. This level of care ensures that the final product is polished and professional, reflecting the brand accurately and effectively. 

In a different context, attention to detail can also be seen in a chef who carefully measures ingredients, garnishes dishes with precision, and ensures that every plate is presented beautifully before serving, enhancing the dining experience.

### Guided Response (Default Prompt):

Attention to detail refers to the ability to notice and consider the small elements that contribute to the overall quality of a task or project. Here’s an example:

Imagine a graphic designer working on a marketing campaign. While creating a poster, they not only focus on the main images and text but also pay close attention to the alignment of elements, the color balance, and the font choices. They ensure that the spacing between letters is consistent, the colors are harmonious, and even the shadows and highlights are applied correctly to enhance the visual appeal. Additionally, they might double-check for any typos or grammatical errors in the text before finalizing the design.

This meticulous approach ensures that the final product is polished and professional, making a strong impression on the audience. Attention to detail like this can significantly elevate the quality of work in any field!

### RAG Response (Default Prompt):

This excerpt provides a fascinating look into the collaborative and detail-oriented approach that Apple employs in its product development, particularly in enhancing the iPhone's camera capabilities. The challenges faced by the camera team in implementing portrait mode highlight the importance of addressing even the rarest of corner cases to maintain high standards of quality—what Apple refers to as "zero artifacts."

The discussions between the camera team and design leaders illustrate how diverse perspectives can lead to innovative solutions. By analyzing the work of great portrait photographers, the teams were able to redefine their quality standards, ultimately achieving a more aesthetically pleasing result that aligns with user expectations.

Moreover, the push for a live preview feature demonstrates Apple's commitment to user experience. The collaboration between different teams, including hardware, software, and human interface design, showcases how cross-functional teamwork can lead to significant advancements in technology.

Apple's philosophy of "accountability without control" is particularly noteworthy. It emphasizes the importance of shared goals and collaboration, even when team members have their own commitments. This approach fosters a culture of innovation while ensuring that everyone is working towards a common objective.

Overall, this case study exemplifies how Apple’s meticulous attention to detail and collaborative spirit contribute to its reputation for producing high-quality products that resonate with users. If you have any questions or want to discuss this further, feel free to ask!


---


In [20]:
# Generate actionable insights and business recommendations using the RAG pipeline
insights_prompt = (
    "Based on the article 'How Apple Is Organized for Innovation', generate 5 concise, actionable insights "
    "that product and engineering leaders can apply. For each insight provide a one-line action and a one-sentence rationale."
)

recommendations_prompt = (
    "Based on the same article, produce 7 concrete business recommendations for senior leadership "
    "and org design to improve innovation outcomes. For each recommendation provide a short rationale and a suggested first step."
)

# Get RAG-backed responses
actionable_insights = getRAGResponse(sysPromptDefault, insights_prompt)
business_recommendations = getRAGResponse(sysPromptDefault, recommendations_prompt)

# Display in notebook
printmd("## Actionable Insights")
printmd(actionable_insights)

printmd("## Business Recommendations")
printmd(business_recommendations)

# Persist results to a JSON file for reuse
output = {
    "actionable_insights": actionable_insights,
    "business_recommendations": business_recommendations
}

## Actionable Insights

It looks like you're referencing an article from the Harvard Business Review titled "How Apple Is Organized for Innovation" by Joel M. Podolny and Morten T. Hansen. This piece explores how Apple's organizational structure fosters a culture of innovation, emphasizing the importance of having experts lead teams of experts. 

The article likely discusses key elements such as collaboration, cross-functional teams, and the role of leadership in nurturing creativity and innovation within the company. Apple's approach to maintaining a dynamic and innovative environment is often cited as a significant factor in its success.

If you're interested in specific insights or themes from the article, feel free to ask! I'm here to help.

## Business Recommendations

The article "How Apple Is Organized for Innovation" by Joel M. Podolny and Morten T. Hansen, published in the Harvard Business Review, delves into the unique organizational structure of Apple Inc. and how it fosters a culture of innovation. 

One of the key insights from the article is that Apple’s approach is centered around having experts lead other experts. This means that teams are often composed of highly skilled individuals who are not only knowledgeable in their respective fields but also empowered to make decisions. This structure encourages collaboration and creativity, allowing for rapid problem-solving and the development of innovative products.

The article also highlights the importance of a strong organizational culture that aligns with Apple's mission and values. This culture promotes a sense of ownership and accountability among employees, which is crucial for driving innovation.

If you're interested in understanding how organizational culture can impact innovation, this article provides valuable insights into Apple's strategies and practices. Just remember, while you can enjoy the article for personal use, sharing or distributing it is not permitted. Happy reading!